# Создание нейронной сети

В этом задании мы создадим полносвязную нейронную сеть используя при этом низкоуровневые механизмы tensorflow.

Архитектутра нейросети представлена на следующем рисунке. Как видите, в ней имеется один входной слой, два скрытых, а так же выходной слой. В качестве активационной функции в скрытых слоях будет использоваться сигмоида. На выходном слое мы используем softmax.

Часть кода по созданию сети уже написана, от вас требуется заполнить пропуски в указанных местах.

## Архитектура нейронной сети

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>


## О датасете MNIST

Данную нейросеть мы будем обучать на датасете MNIST. Этот датасет представляет собой большое количество изображений рукописных цифр размером $28 \times 28$ пикселей. Каждый пиксель принимает значение от 0 до 255.

Как и раньше датасет будет разеделен на обучающую и тестовую выборки. При этом мы выполним нормализацию всех изображений, чтобы значения пикселей находились в промежутке от 0 до 1, разделив яркость каждого пикселя на 255.

Кроме того, архитектура нейронной сети ожидает на вход вектор. В нашем же случае каждый объект выборки представляет собой матрицу. Что же делать? В этом задании мы "растянем" матрицу $28 \times 28$, получив при этом вектор, состоящей из 784 элементов.

![MNIST Dataset](https://www.researchgate.net/profile/Steven-Young-5/publication/306056875/figure/fig1/AS:393921575309346@1470929630835/Example-images-from-the-MNIST-dataset.png)

Больше информации о датасете можно найти [здесь](http://yann.lecun.com/exdb/mnist/).

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
num_classes = 10  # загальна кількість класів, у нашому випадку це цифри від 0 до 9
num_features = 784  # кількість атрибутів вхідного вектора 28 * 28 = 784

learning_rate = 0.001  # швидкість навчання нейронної мережі
training_steps = 3000  # максимальна кількість епох
batch_size = 256  # перераховувати ваги мережі ми будемо не на усій вибірці, а на її випадковій підмножині розміром batch_size
display_step = 100  # кожні 100 ітерацій ми будемо показувати поточне значення функції втрат та точності

n_hidden_1 = 128  # кількість нейронів у 1-ому шарі
n_hidden_2 = 256  # кількість нейронів у 2-ому шарі


In [3]:
from tensorflow.keras.datasets import mnist

# Завантажуємо датасет
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Перетворюємо цілі числові пікселі у тип float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Перетворюємо матриці розміром 28x28 пікселів у вектор з 784 елементів
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Нормалізуємо значення пікселів
x_train, x_test = x_train / 255., x_test / 255.

# Перемішуємо тренувальні дані
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)


In [4]:
class DenseLayer(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.weights = tf.Variable(tf.random.normal([in_features, out_features]))
        self.bias = tf.Variable(tf.zeros([out_features]))

    def __call__(self, x):
        linear = tf.matmul(x, self.weights) + self.bias
        return tf.nn.relu(linear)

class NN(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        # Первий прихований шар, що складається з 128 нейронів
        self.layer1 = DenseLayer(num_features, n_hidden_1, name="Layer1")

        # Другий прихований шар, що складається з 256 нейронів
        self.layer2 = DenseLayer(n_hidden_1, n_hidden_2, name="Layer2")

        # Вихідний шар
        self.out_layer = DenseLayer(n_hidden_2, num_classes, name="OutLayer")

    def __call__(self, x):
        # Подача вхідного вектора через перший прихований шар
        layer1_out = self.layer1(x)

        # Подача виходу першого прихованого шару в другий прихований шар
        layer2_out = self.layer2(layer1_out)

        # Подача виходу другого прихованого шару в вихідний шар
        out_layer_out = self.out_layer(layer2_out)

        # Зверніть увагу, що функцію softmax ми використовуємо для того, щоб вивід
        # нейронної мережі мав значення від 0 до 1, відповідно ймовірності належності вхідного
        # об'єкта до одного з 10 класів

        x = tf.nn.softmax(out_layer_out)
        return x


In [5]:
# Функція для обчислення кросс-ентропії
def cross_entropy(y_pred, y_true):
    # Кодуємо мітку у вигляді one-hot вектора.
    y_true = tf.one_hot(y_true, depth=num_classes)

    # Обмежуємо значення передбачення, щоб уникнути помилки log(0).
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Обчислюємо кросс-ентропію
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Функція для обчислення точності
def accuracy(y_pred, y_true):
    # Порівнюємо передбачені класи зі справжніми класами
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    
    # Обчислюємо середню точність в наборі даних
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [6]:
# Створюємо екземпляр нейронної мережі
neural_net = NN(name="mnist")

# Функція навчання нейромережі
def train(nn, dataset, num_epochs):
    # Для підгонки ваг використовуємо стохастичний градієнтний спуск
    optimizer = tf.optimizers.SGD(learning_rate)

    for epoch in range(num_epochs):
        for batch_x, batch_y in dataset:
            # Включаємо автоматичне диференціювання
            with tf.GradientTape() as g:
                pred = nn(batch_x)
                loss = cross_entropy(pred, batch_y)

                # Створюємо список оптимізованих параметрів
                trainable_variables = nn.trainable_variables

                # Обчислюємо градієнт відносно параметрів
                gradients = g.gradient(loss, trainable_variables)

                # Модифікуємо параметри відповідно до градієнту
                optimizer.apply_gradients(zip(gradients, trainable_variables))

        if (epoch + 1) % display_step == 0:
            pred = nn(x_train)
            loss = cross_entropy(pred, y_train)
            acc = accuracy(pred, y_train)
            print(f"Epoch {epoch+1}, Loss: {loss}, Accuracy: {acc}")

# Виклик функції навчання зі зазначенням кількості ітерацій
train(neural_net, train_data, num_epochs=training_steps)


KeyboardInterrupt: 

In [ ]:
# Тренування мережі

loss_history = []  # кожні display_step кроків зберігайте в цьому списку поточну помилку нейромережі
accuracy_history = [] # кожні display_step кроків зберігайте в цьому списку поточну точність нейромережі

# У цьому циклі ми будемо проводити навчання нейронної мережі
# з тренувального датасету train_data вийміть випадкову підмножину, на якій
# відбудеться тренування. Використовуйте метод take, доступний для тренувального датасету.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps)):
    # Оновлюємо ваги нейронної мережі
    train(neural_net, batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neural_net(batch_x)
        current_loss = cross_entropy(pred, batch_y)
        loss_history.append(current_loss)
        
        current_accuracy = accuracy(pred, batch_y)
        accuracy_history.append(current_accuracy)
        print(f"Step: {step}, Loss: {current_loss}, Accuracy: {current_accuracy}")

In [ ]:
import matplotlib.pyplot as plt

# Виведення графіків
plt.figure(figsize=(12, 6))

# Графік залежності втрат від кроку навчання
plt.subplot(1, 2, 1)
plt.plot(range(display_step, training_steps + 1, display_step), loss_history, marker='o')
plt.title('Зміна втрат від кроку навчання')
plt.xlabel('Крок навчання')
plt.ylabel('Втрати')

# Графік залежності точності від кроку навчання
plt.subplot(1, 2, 2)
plt.plot(range(display_step, training_steps + 1, display_step), accuracy_history, marker='o', color='orange')
plt.title('Зміна точності від кроку навчання')
plt.xlabel('Крок навчання')
plt.ylabel('Точність')

plt.tight_layout()
plt.show()


In [ ]:
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.batch(batch_size)

total_correct = 0
total_samples = 0

# Пройдемося по тестовим даним
for batch_x, batch_y in test_data:
    pred = neural_net(batch_x)
    batch_y = tf.cast(batch_y, tf.int64)  # Перетворюємо тип даних batch_y на int64
    correct = tf.equal(tf.argmax(pred, 1), batch_y)
    total_correct += tf.reduce_sum(tf.cast(correct, tf.int32))
    total_samples += batch_x.shape[0]

# Обчислюємо точність на тестових даних
test_accuracy = total_correct / total_samples
print("Точність на тестових даних: {:.4f}".format(test_accuracy.numpy()))


In [ ]:
import random

# Випадково виберемо 5 індексів з тестових даних
random_indices = random.sample(range(len(x_test)), 5)

# Збережемо вибрані зображення та їх класи
selected_images = x_test[random_indices]
selected_labels = y_test[random_indices]

# Прогнозуємо класи для вибраних зображень
predictions = neural_net(selected_images)
predicted_classes = tf.argmax(predictions, axis=1)

# Візуалізуємо результати
plt.figure(figsize=(15, 5))

for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(selected_images[i].reshape(28, 28), cmap='gray')
    plt.title(f'Predicted: {predicted_classes[i].numpy()}, Actual: {selected_labels[i]}')
    plt.axis('off')

plt.tight_layout()
plt.show()
